<a href="https://colab.research.google.com/github/hana-magdy/Face-Recognition/blob/main/FaceRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports:


In [1]:
from PIL import Image
from numpy import asarray
from sklearn.metrics import accuracy_score
import numpy as np
import glob
import cv2
import zipfile
import  os
from scipy.spatial import distance
from sklearn.neighbors import KNeighborsClassifier
from tabulate import tabulate
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
zip_ref = zipfile.ZipFile("archive (1).zip", "r")
zip_ref.extractall()
zip_ref.close()

In [3]:
 
y = []
Data_matrix = []
for i in range(1,41):
  folder = "s"
  folder+=str(i)
  y.append(i)
  files = glob.glob (folder)
  for j in range(1,11):
      file_name = folder + "/"+str(j)+".pgm"
      img = Image.open(file_name)
      data = np.array(img)
      print(data)
      x = data.flatten()
      Data_matrix.append(x)
      print(x.shape)
      print("Path")
      print(file_name)
      j=j+1
  j = 1
  i = i +1

[[48 49 45 ... 56 56 54]
 [45 52 39 ... 52 50 51]
 [45 50 42 ... 48 53 50]
 ...
 [50 48 50 ... 45 46 46]
 [45 54 49 ... 46 47 47]
 [51 51 51 ... 47 46 46]]
(10304,)
Path
s1/1.pgm
[[ 60  60  62 ...  40  53  46]
 [ 58  73  54 ...  34  50  51]
 [ 68  77  80 ...  36  50  49]
 ...
 [ 24 143 160 ...  34  33  33]
 [ 18 136 161 ...  36  36  31]
 [  9 111 168 ...  32  34  34]]
(10304,)
Path
s1/2.pgm
[[39 44 53 ... 47 48 47]
 [44 44 61 ... 47 45 46]
 [59 39 57 ... 46 46 44]
 ...
 [49 46 42 ... 30 32 28]
 [46 35 34 ... 30 29 27]
 [32 37 27 ... 29 26 29]]
(10304,)
Path
s1/3.pgm
[[63 53 35 ... 36 44 45]
 [56 48 31 ... 57 50 50]
 [52 29 35 ... 54 55 61]
 ...
 [35 37 36 ... 44  9 25]
 [36 35 37 ... 40  7 28]
 [35 38 34 ... 41 10 24]]
(10304,)
Path
s1/4.pgm
[[64 76 80 ... 52 54 52]
 [60 61 60 ... 52 53 51]
 [63 80 79 ... 53 52 59]
 ...
 [21 22 12 ... 37 42 35]
 [19 20  7 ... 42 35 42]
 [24 14 11 ... 35 37 39]]
(10304,)
Path
s1/5.pgm
[[ 43  50  41 ...  55  55  56]
 [ 45  53  44 ...  67  62  52]
 [ 57  

In [4]:
Data_matrix = np.array(Data_matrix)
print("Total data:")
print(Data_matrix.shape)
print(Data_matrix)

print("Labels:")
print(y)


Total data:
(400, 10304)
[[ 48  49  45 ...  47  46  46]
 [ 60  60  62 ...  32  34  34]
 [ 39  44  53 ...  29  26  29]
 ...
 [125 119 124 ...  36  39  40]
 [119 120 120 ...  89  94  85]
 [125 124 124 ...  36  35  34]]
Labels:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]


splitting the data:


In [37]:
Testing =  Data_matrix[::2]
Training = Data_matrix[1::2]
print("Training")
print(Training)

print("Testing")
print(Testing)

Training_Labels=[]
Original_Testing_Labels=[]
for i in range(1,41):
  for j in range(1,6):
    Training_Labels.append(i)
    Original_Testing_Labels.append(i)

print("Training Labels:")
print(Training_Labels)

Training
[[ 60  60  62 ...  32  34  34]
 [ 63  53  35 ...  41  10  24]
 [ 43  50  41 ... 158 153 169]
 ...
 [123 121 126 ...  40  35  42]
 [125 119 124 ...  36  39  40]
 [125 124 124 ...  36  35  34]]
Testing
[[ 48  49  45 ...  47  46  46]
 [ 39  44  53 ...  29  26  29]
 [ 64  76  80 ...  35  37  39]
 ...
 [128 125 125 ...  85  90  84]
 [129 127 133 ...  93  93  93]
 [119 120 120 ...  89  94  85]]
Training Labels:
[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32

### **PCA function:**


In [32]:
MeanMatrix=np.array(Training).mean(axis=0)
print("Mean Matrix: ")
print(MeanMatrix)
print("Mean Matrix shape: ")
print(MeanMatrix.shape)
print("\n *********************************** \n")
print("Centred Data: ")
Centred_Data=np.array(Training-MeanMatrix)
print(Centred_Data)
print("\n *********************************** \n")
print("Covariance matrix: ")
Covariance_Matrix=np.cov(Centred_Data.T,bias=True)
print(Covariance_Matrix)
print("\n *********************************** \n")
Eigenvalues,EigenVectors=np.linalg.eigh(Covariance_Matrix)
print("Eigenvalues: ")
print(Eigenvalues)
print("\n *********************************** \n")
print("Eigenvectors: ")
print(EigenVectors)
print("\n *********************************** \n")
print("Sum of total EigenValues")
SumEigenValues=Eigenvalues.sum()
print(SumEigenValues)
idx = np.argsort(Eigenvalues)[::-1]
Sorted_eigenValues= Eigenvalues[idx]
#Sorted_eigenValues=np.diagflat(Sorted_eigenValues)
Sorted_eigenvectors= EigenVectors[:,idx]


Mean Matrix: 
[85.12  84.89  85.165 ... 77.24  74.335 73.37 ]
Mean Matrix shape: 
(10304,)

 *********************************** 

Centred Data: 
[[-25.12  -24.89  -23.165 ... -45.24  -40.335 -39.37 ]
 [-22.12  -31.89  -50.165 ... -36.24  -64.335 -49.37 ]
 [-42.12  -34.89  -44.165 ...  80.76   78.665  95.63 ]
 ...
 [ 37.88   36.11   40.835 ... -37.24  -39.335 -31.37 ]
 [ 39.88   34.11   38.835 ... -41.24  -35.335 -33.37 ]
 [ 39.88   39.11   38.835 ... -41.24  -39.335 -39.37 ]]

 *********************************** 

Covariance matrix: 
[[1318.8756   1299.2282   1304.6652   ... -235.7588   -126.0752
   -75.5394  ]
 [1299.2282   1295.5879   1293.30315  ... -218.1036   -112.70315
   -64.3043  ]
 [1304.6652   1293.30315  1306.107775 ... -216.9946    -99.325275
   -49.28605 ]
 ...
 [-235.7588   -218.1036   -216.9946   ... 2505.0524   1995.4546
  1882.7162  ]
 [-126.0752   -112.70315   -99.325275 ... 1995.4546   1972.392775
  1859.56105 ]
 [ -75.5394    -64.3043    -49.28605  ... 1882.7162  

In [143]:
def PCA(Alpha):
  FractionalSum=0 
  FractionalVariance=0
  index=0
  for i in range(0,len(Sorted_eigenValues)):
    FractionalVariance=FractionalSum / SumEigenValues
    if  FractionalVariance < Alpha :
      index=i
      FractionalSum+=Sorted_eigenValues[i]
    else:
      break
 
  print("Fractional Variance")
  print(FractionalVariance)
  print("r= ",index)
          

  print("\n *********************************** \n")
  Reduced_Eigenvectors=Sorted_eigenvectors[:, 0:index+1]
  print("Reduced EigenVector= ")
  print(Reduced_Eigenvectors)
  print(Reduced_Eigenvectors.shape)
  print("\n *********************************** \n")

  Reduced_EigenvectorsT=np.transpose(Reduced_Eigenvectors)
  print("projection matrix= ")
  print(Reduced_EigenvectorsT) 

  print("\n *********************************** \n")
  print("Projected Training matrix = ")
  projected_training=np.dot(Centred_Data,Reduced_Eigenvectors)
  print(projected_training)

  print("\n *********************************** \n")
  Temp_Testing=np.subtract(Testing,Testing.mean(axis=0))
  print("Testing",Testing)
  print("Temp_Testing",Temp_Testing)
  projected_testing=np.dot(Temp_Testing,Reduced_Eigenvectors)
  print("Projected Testing matrix = ")
  print(projected_testing)
  print("\n *********************************** \n")
 
  return projected_training,projected_testing

In [144]:
print("PCA output for Alpha=0.8 :")
projected_training08,projected_testing08=PCA(0.8)

PCA output for Alpha=0.8 :
Fractional Variance
0.8029455554165238
r=  36

 *********************************** 

Reduced EigenVector= 
[[ 0.00124556 -0.01536171 -0.01891079 ...  0.00118506 -0.00548698
   0.00018831]
 [ 0.00126454 -0.01513146 -0.01911777 ... -0.0006894  -0.0047121
   0.00122833]
 [ 0.00156465 -0.01516195 -0.01903942 ...  0.00010793 -0.00319525
   0.00043875]
 ...
 [ 0.00873725  0.00961112 -0.01287451 ... -0.00532322  0.00449344
  -0.00201685]
 [ 0.00721705  0.0077597  -0.01377318 ...  0.00838453  0.00849715
  -0.00858265]
 [ 0.0084008   0.00692708 -0.01438815 ...  0.00716891  0.00831642
  -0.00863673]]
(10304, 37)

 *********************************** 

projection matrix= 
[[ 0.00124556  0.00126454  0.00156465 ...  0.00873725  0.00721705
   0.0084008 ]
 [-0.01536171 -0.01513146 -0.01516195 ...  0.00961112  0.0077597
   0.00692708]
 [-0.01891079 -0.01911777 -0.01903942 ... -0.01287451 -0.01377318
  -0.01438815]
 ...
 [ 0.00118506 -0.0006894   0.00010793 ... -0.00532322  

In [145]:
print("PCA output for Alpha=0.85 :")
projected_training085,projected_testing085=PCA(0.85)

PCA output for Alpha=0.85 :
Fractional Variance
0.8523113375505037
r=  52

 *********************************** 

Reduced EigenVector= 
[[ 0.00124556 -0.01536171 -0.01891079 ... -0.00658215  0.0063633
  -0.00188005]
 [ 0.00126454 -0.01513146 -0.01911777 ... -0.00497546  0.00626057
  -0.00089457]
 [ 0.00156465 -0.01516195 -0.01903942 ... -0.00530911  0.00690086
  -0.00163866]
 ...
 [ 0.00873725  0.00961112 -0.01287451 ... -0.00490491 -0.01032198
  -0.00532878]
 [ 0.00721705  0.0077597  -0.01377318 ...  0.0029665  -0.00741123
  -0.00191385]
 [ 0.0084008   0.00692708 -0.01438815 ... -0.00282685 -0.00728419
  -0.00833635]]
(10304, 53)

 *********************************** 

projection matrix= 
[[ 0.00124556  0.00126454  0.00156465 ...  0.00873725  0.00721705
   0.0084008 ]
 [-0.01536171 -0.01513146 -0.01516195 ...  0.00961112  0.0077597
   0.00692708]
 [-0.01891079 -0.01911777 -0.01903942 ... -0.01287451 -0.01377318
  -0.01438815]
 ...
 [-0.00658215 -0.00497546 -0.00530911 ... -0.00490491 

In [146]:
print("PCA output for Alpha=0.9:")
projected_training09,projected_testing09=PCA(0.9)

PCA output for Alpha=0.9:
Fractional Variance
0.9007600309473764
r=  76

 *********************************** 

Reduced EigenVector= 
[[ 0.00124556 -0.01536171 -0.01891079 ...  0.00179702 -0.0038579
   0.00149348]
 [ 0.00126454 -0.01513146 -0.01911777 ...  0.00188084 -0.00483295
   0.00235076]
 [ 0.00156465 -0.01516195 -0.01903942 ...  0.00152641 -0.0040227
   0.00128325]
 ...
 [ 0.00873725  0.00961112 -0.01287451 ...  0.0070125  -0.02963997
   0.0032255 ]
 [ 0.00721705  0.0077597  -0.01377318 ...  0.01557117 -0.02348247
   0.01523977]
 [ 0.0084008   0.00692708 -0.01438815 ...  0.00818988 -0.01151255
   0.00688259]]
(10304, 77)

 *********************************** 

projection matrix= 
[[ 0.00124556  0.00126454  0.00156465 ...  0.00873725  0.00721705
   0.0084008 ]
 [-0.01536171 -0.01513146 -0.01516195 ...  0.00961112  0.0077597
   0.00692708]
 [-0.01891079 -0.01911777 -0.01903942 ... -0.01287451 -0.01377318
  -0.01438815]
 ...
 [ 0.00179702  0.00188084  0.00152641 ...  0.0070125   0.

In [147]:
print("PCA output for Alpha=0.95 :")
projected_training095,projected_testing095=PCA(0.95)

PCA output for Alpha=0.95 :
Fractional Variance
0.9500861567347986
r=  115

 *********************************** 

Reduced EigenVector= 
[[ 0.00124556 -0.01536171 -0.01891079 ... -0.0105497  -0.00116262
  -0.00499452]
 [ 0.00126454 -0.01513146 -0.01911777 ... -0.0063317  -0.00192066
  -0.00479632]
 [ 0.00156465 -0.01516195 -0.01903942 ... -0.00767535 -0.00365636
  -0.00746669]
 ...
 [ 0.00873725  0.00961112 -0.01287451 ...  0.00414923  0.00148975
  -0.01489488]
 [ 0.00721705  0.0077597  -0.01377318 ... -0.00193868 -0.0010741
  -0.02912841]
 [ 0.0084008   0.00692708 -0.01438815 ... -0.0059235  -0.00401374
  -0.038607  ]]
(10304, 116)

 *********************************** 

projection matrix= 
[[ 0.00124556  0.00126454  0.00156465 ...  0.00873725  0.00721705
   0.0084008 ]
 [-0.01536171 -0.01513146 -0.01516195 ...  0.00961112  0.0077597
   0.00692708]
 [-0.01891079 -0.01911777 -0.01903942 ... -0.01287451 -0.01377318
  -0.01438815]
 ...
 [-0.0105497  -0.0063317  -0.00767535 ...  0.0041492

In [148]:
def Label_prediction(k,ProjectedTraining , ProjectedTesting):

  model = KNeighborsClassifier(n_neighbors=k)
  model.fit(ProjectedTraining,np.array(Training_Labels))
  Tested_Labels= model.predict(ProjectedTesting)

  return Tested_Labels

In [149]:
print("Tested labels for 0.8=")
Tested_Labels08=Label_prediction (1,projected_training08,projected_testing08)
print(Tested_Labels08)

print("\n *********************************** \n")

print("Tested labels for 0.85=")
Tested_Labels085=Label_prediction (1,projected_training085,projected_testing085)
print(Tested_Labels085)

print("\n *********************************** \n")

print("Tested labels for 0.9=")
Tested_Labels09=Label_prediction (1,projected_training09,projected_testing09)
print(Tested_Labels09)

print("\n *********************************** \n")

print("Tested labels for 0.95=")
Tested_Labels095=Label_prediction (1,projected_training095,projected_testing095)
print(Tested_Labels095)

Tested labels for 0.8=
[16  2  1  1  1  2  2  2  2  2  3  3  3  3  3  4  4  4  4  4  5  5  5  5
  5  6  6  6  6  6  7  7  7  7  7  8  8  8  8  8  9  9  9  9  9 10 10 10
 10 38 11 11 11 11 11 12 12 12 12 12 13 13 13 13 13 14 14 14 14 14 15 15
 15 15 15 16 16 16 16 16 17 17 17 17 17 18 18 18 18 18 19 19 19 19 36 20
 20  3 20 20 21 21 21 21 21 22 22 22 22 22 23 23 23 23 23 24 24 24 24 24
 25 25 25 25 25 26 26 26 26 26 27 27 27 27 27 28 28 28 28 28 29 29 29 29
 29 30 30 30 30 30 31 31 31 21 31 32 32 32  2 32 33 33 33 33 33 34 34 34
 34 34 40 15 35 35 35  7 36  7 36 36 37 37 37 37 37 38 38 38 38 38 39 39
 39 39 39 40 40  5  5  5]

 *********************************** 

Tested labels for 0.85=
[16  2  1  1  1  2  2  2  2  2  3  3  3  3  3  4  4  4  4  4  5  5  5  5
  5  6  6  6  6  6  7  7  7  7  7  8  8  8  8  8  9  9  9  9  9 10 10 10
 10 38 11 11 11 11 11 12 12 12 12 12 13 13 13 13 13 14 14 14 14 14 15 15
 15 15 15 16 16 16 16 16 17 17 17 17 17 18 18 18 18 18 19 19 19 19 36 20
 20  3 20 2

In [150]:
 print("Accuracy of 0.8 =")
 Accuracy08 = accuracy_score(Original_Testing_Labels, Tested_Labels08)
 print(Accuracy08*100,"%")

 print("Accuracy of 0.85 =")
 Accuracy085 = accuracy_score(Original_Testing_Labels, Tested_Labels085)
 print(Accuracy085*100,"%")  

 print("Accuracy of 0.9 =")
 Accuracy09 = accuracy_score(Original_Testing_Labels, Tested_Labels09)
 print(Accuracy09*100,"%")

 print("Accuracy of 0.95 =")
 Accuracy095= accuracy_score(Original_Testing_Labels, Tested_Labels095)
 print(Accuracy095*100,"%")

Accuracy of 0.8 =
93.0 %
Accuracy of 0.85 =
93.5 %
Accuracy of 0.9 =
94.0 %
Accuracy of 0.95 =
93.5 %



### **LDA**





In [172]:
#1
ListOfClasses=[]
for i in range(0,40):
  ListOfClasses.append(Training[5*i:5*(i+1),:])

  
  
#print(ListOfClasses)
#2
ListOfMeans=[]
for i in range(0,40):
  ListOfMeans.append(np.array(ListOfClasses[i]).mean(axis=0))

#print(len(ListOfMeans))  
#print(ListOfMeans)

#3
OverallMean=np.array(Training).mean(axis=0)
print(OverallMean)
Sb=np.zeros((10304,10304))
for i in range(0,40):
  expression=(np.outer((ListOfMeans[i]-OverallMean),np.array((ListOfMeans[i]-OverallMean)).T))
  Sb=Sb+ expression
Sb=5*Sb  
print(Sb)

#4
ListOfCentredData=[]
for i in range(0,40):
  ListOfCentredData.append(ListOfClasses[i]-np.array(ListOfMeans[i]).T)

#print(ListOfCentredData)

#5
ListOfScatterMatrcies=np.zeros((10304,10304))
for j in range(0,40):
 expression=np.dot(np.array(ListOfCentredData[j]).T,np.array(ListOfCentredData[j]))
 ListOfScatterMatrcies=ListOfScatterMatrcies+expression
   
print(ListOfScatterMatrcies.shape)
print(ListOfScatterMatrcies)




[array([[ 60,  60,  62, ...,  32,  34,  34],
       [ 63,  53,  35, ...,  41,  10,  24],
       [ 43,  50,  41, ..., 158, 153, 169],
       [ 44,  43,  32, ...,  43,  43,  37],
       [ 34,  34,  33, ...,  37,  40,  33]], dtype=uint8), array([[ 37,  35,  35, ...,  26,  28,  28],
       [ 34,  36,  35, ...,  32,  25,  25],
       [ 34,  35,  35, ...,  31,  24,  24],
       [ 34,  39,  35, ..., 135, 138, 148],
       [ 37,  34,  38, ..., 135, 143, 133]], dtype=uint8), array([[100,  99, 108, ...,  44,  42,  44],
       [105, 102, 108, ...,  42,  35,  37],
       [110, 106, 109, ...,  49,  53,  53],
       [107, 104, 109, ...,  58,  56,  57],
       [104, 109, 103, ...,  57,  56,  59]], dtype=uint8), array([[116, 112, 115, ...,  51,  55,  54],
       [119, 121, 119, ...,  82,  78,  80],
       [121, 123, 121, ...,  88,  83,  78],
       [118, 114, 117, ...,  58,  54,  52],
       [119, 124, 118, ...,  75,  77,  82]], dtype=uint8), array([[130, 132, 129, ...,  53,  50,  52],
       [130, 13

In [16]:
#6
inv=np.linalg.inv(ListOfScatterMatrcies)
s_1_b=np.dot(inv,Sb)
#print(s_1_b)
EigenValues,EigenVector=np.linalg.eigh(s_1_b)
print(EigenValues.shape)
print(EigenVector.shape)



(10304,)
(10304, 10304)


In [177]:
idx = np.argsort(EigenValues)[::-1]
Sorted_eigenValues= EigenValues[idx]
Sorted_eigenvectors= EigenVector[:,idx]

Reduced_Dimensions=Sorted_eigenvectors[0:39]


LDAProjectionMatrix=Reduced_Dimensions.T
print(LDAProjectionMatrix)


[[-0.00280294 -0.00297361 -0.00268792 ... -0.0125448  -0.01112841
  -0.01234817]
 [ 0.01930769  0.01894665  0.01919557 ...  0.01406648  0.01590946
   0.01829934]
 [-0.01114684 -0.011058   -0.01164946 ... -0.00040893 -0.00026298
  -0.00133917]
 ...
 [ 0.01079212  0.01078261  0.01106048 ...  0.00113775 -0.00136213
  -0.00106431]
 [-0.01908478 -0.01884978 -0.01862514 ... -0.01696852 -0.01328085
  -0.0135706 ]
 [-0.00276778 -0.00294529 -0.00263768 ... -0.01291808 -0.01076931
  -0.01165422]]


In [178]:
LDAProjectedTraining=np.dot(Training,LDAProjectionMatrix)
print(LDAProjectedTraining)
print(LDAProjectedTraining.shape)
LDAProjectedTesting=np.dot(Testing,LDAProjectionMatrix)
print(LDAProjectedTesting)
print(LDAProjectedTesting.shape)

[[ -15.60972121  125.21639407  -91.37748762 ...   87.96089121
    19.37719225  226.73292103]
 [ -29.08127065  124.20658812 -117.98656256 ...  123.98625516
    70.70574197  159.35714784]
 [ -25.71139757  112.64179747 -145.73779512 ...  121.84076951
    34.29593793  160.20323107]
 ...
 [  12.64733713  115.9333482   -80.16642847 ...   50.3549239
    25.07714684  126.01364473]
 [   1.98450013  110.7373238   -80.48556483 ...   27.1354228
     3.71371941  129.4637805 ]
 [   7.12468105  104.27538408  -66.09413017 ...   77.9072423
    -0.36793209  128.02032243]]
(200, 39)
[[  -6.64691607   98.9764722  -117.83366916 ...   79.50526491
   -15.2381256   162.28287966]
 [ -37.84809037   88.47422213 -118.3081307  ...  126.50964298
    29.96667997  162.83501336]
 [ -72.90780693   78.97534308  -31.15296754 ...  124.39618337
    36.18716884  206.33741606]
 ...
 [   3.17264349   94.23676427  -59.72764023 ...   33.4396377
   -33.41123029  141.04903333]
 [ -40.50325749   51.42997351  -64.50661574 ...   18.

In [179]:
LDAPredictedLabels=Label_prediction(1,LDAProjectedTraining,LDAProjectedTesting)
print(LDAPredictedLabels)


[16  1  1  2  1  2  2  2  2  2 29  3  3  3  3  4  4  3  4  4  5  5  5  5
  5  6  6  6  6  6  7  7  7  7  7  8  8  8  8  8  9  9 30  9  9 10 10 10
 10 10  4 36 11 11 34 12 12 12 12 12 13 13 13 13 13 14 14 14 14 14 15 15
 15 15 15 28 16 16 16  2 17 17 17 17 17 18 18 18 18 18 19 19 19 19 36 20
 29 29 20 20 30 29 21 29 21 22 22 29 22 22 23 23 29 23 29 24 24 16 24 24
 15 25 25 25 25 26 26 26 28 26 27 27 27  8 27 28 28 28 14 28 29 29 29 29
 29 30 30 30 30 29 21 31 31 29 21 24 32  2  7 32 30 33 21 33 30 34 34 34
 34 34 40  5 35  4 35  7 36  7 36 17 37 37 37 37 37 38 38 38 38  9 39 39
 30 39 39  5 40 15 26  5]


In [180]:
LDA_Accuracy1 = accuracy_score(Original_Testing_Labels, LDAPredictedLabels)
print("Accuracy1 = ",LDA_Accuracy1*100, "%")

Accuracy1 =  77.0 %


### **Classfier tunning**

In [21]:
AccuracyLDA=[]

AccuracyLDA.append([1,LDA_Accuracy1])

LDAPredictedLabels3=Label_prediction(3,LDAProjectedTraining,LDAProjectedTesting)
LDA_Accuracy3 = accuracy_score(Original_Testing_Labels, LDAPredictedLabels3)
AccuracyLDA.append([3,LDA_Accuracy3])
print(LDAPredictedLabels3)

print("\n *********************************** \n")

LDAPredictedLabels5=Label_prediction(5,LDAProjectedTraining,LDAProjectedTesting)
LDA_Accuracy5 = accuracy_score(Original_Testing_Labels, LDAPredictedLabels5)
AccuracyLDA.append([5,LDA_Accuracy5])
print(LDAPredictedLabels5)

print("\n *********************************** \n")

LDAPredictedLabels7=Label_prediction(7,LDAProjectedTraining,LDAProjectedTesting)
LDA_Accuracy7 = accuracy_score(Original_Testing_Labels, LDAPredictedLabels7)
AccuracyLDA.append([7,LDA_Accuracy7])
print(LDAPredictedLabels7)



[ 2  1  1  2  1  2  2  2  2 37  3  3  3  3  3  4  4  4  4  4  5  5  5  5
  5  5  5  6  6  6  7  7  7  7  7  8  8  8  8  8  9  9  9  9  9 10 10 10
 10 10  4 27 11 11 34 12 12 12 12 12 13 13 13 13 13 14 14 14 14 14 15 15
  5  5 15 11 24  1 16  2  4 29 17 17 17 18  5 18 18 18 19 19 19 19 11 20
 29  3 16 20 30 29 21 21 21 22 22 22 22  2 23 23 23 23 23 16 24  2 24 24
  4 25 25 25 25 26  5 26 28 26 27 27 27 27 27 13 28 14 14 28 29 29 29 29
 23 30 30 30 30 29 21 31 31 20 21 24 32  2  7 32 30 33 21 21 30 34 34 34
 34 34 21  4 12  4  5  7  7  7 36 17 37 37 37 25 37 38 38 38 38  9 15 22
 30 39 39  5 40  5  5 26]

 *********************************** 

[ 2  1  1  2  1  2  2  2  2 37  3  3 25  3  3  4  4  4  4  4  5  5  5  5
  5  6  6  6  6  6  7  7  7  7  7  8  8  8  8  8  9 30  9  9  9 10 10 10
 10 10  4 27  4 11 34 12 12 12 12 12 13 13 13 13 13 14 37 14 14 14 15 15
  5 15  5  7 24  2 16  2  4 29 17 17 17 18  5 18  5 18 19 19 19 19  4 20
 29  3 20 20 30 29 21 17 21 22 22 22 22  2 23 29 23 23 23 

In [22]:

print (tabulate(AccuracyLDA, headers=["K", "Accuracy"]))


  K    Accuracy
---  ----------
  1       0.77
  3       0.69
  5       0.665
  7       0.63


In [23]:
PCAPredictedLabels08=[]
PCAPredictedLabels085=[]
PCAPredictedLabels09=[]
PCAPredictedLabels095=[]
PCA_Accuracy08=[]
PCA_Accuracy085=[]
PCA_Accuracy09=[]
PCA_Accuracy095=[]
K=[1,3,5,7]
for i in range(0,4):
  PCAPredictedLabels08.append(Label_prediction(K[i],projected_training08,projected_testing08))
  PCAPredictedLabels085.append(Label_prediction(K[i],projected_training085,projected_testing085))
  PCAPredictedLabels09.append(Label_prediction(K[i],projected_training09,projected_testing09))
  PCAPredictedLabels095.append(Label_prediction(K[i],projected_training095,projected_testing095))
for i in range(0,4):
  PCA_Accuracy08.append([K[i],accuracy_score(Original_Testing_Labels, PCAPredictedLabels08[i])])
  PCA_Accuracy085.append([K[i],accuracy_score(Original_Testing_Labels, PCAPredictedLabels085[i])])
  PCA_Accuracy09.append([K[i],accuracy_score(Original_Testing_Labels, PCAPredictedLabels09[i])])
  PCA_Accuracy095.append([K[i],accuracy_score(Original_Testing_Labels, PCAPredictedLabels095[i])])





print("Classifying accuracy with respect to k for PCA (Alpha=0.8) \n")
print (tabulate(PCA_Accuracy08, headers=["K", "Accuracy"]))

print("\n Classifying accuracy with respect to k for PCA (Alpha=0.85) \n")
print (tabulate(PCA_Accuracy085, headers=["K", "Accuracy"]))

print("\n Classifying accuracy with respect to k for PCA (Alpha=0.9) \n")
print (tabulate(PCA_Accuracy09, headers=["K", "Accuracy"]))

print("\n Classifying accuracy with respect to k for PCA (Alpha=0.95) \n")
print (tabulate(PCA_Accuracy095, headers=["K", "Accuracy"]))

Classifying accuracy with respect to k for PCA (Alpha=0.8) 

  K    Accuracy
---  ----------
  1       0.93
  3       0.855
  5       0.805
  7       0.78

 Classifying accuracy with respect to k for PCA (Alpha=0.85) 

  K    Accuracy
---  ----------
  1       0.94
  3       0.855
  5       0.83
  7       0.775

 Classifying accuracy with respect to k for PCA (Alpha=0.9) 

  K    Accuracy
---  ----------
  1       0.945
  3       0.85
  5       0.815
  7       0.755

 Classifying accuracy with respect to k for PCA (Alpha=0.95) 

  K    Accuracy
---  ----------
  1       0.935
  3       0.845
  5       0.815
  7       0.74


### **Non-Faces/Faces**

In [181]:
img = Image.open('/content/dog (3).pgm')     
data = np.array(img)     
print(data.shape)     
x = data.flatten()         
print(x.shape)
  

(92, 112)
(10304,)
